In [1]:
import numpy as np
import scipy.special
import cvxpy as cp
from scipy.optimize import minimize

In [2]:
M = 3 # number of cells
K = 3 # number of users
rho = 1 # max power
BW_max = 6 # max BW, M_hat in paper
rng = np.random.default_rng(2703)
#eta = np.random.rand(K) # power control fraction for each user
#alpha = np.random.rand(M,K)
#beta = np.random.rand(M,K)
eta = rng.random(K) # power control fraction for each user TO BE OTPIMIZED
alpha = rng.random((M,K))
beta = rng.random((M,K))

In [3]:
X_mat = rng.integers(0,2,(M,K))
x_vec = X_mat.flatten('F')

In [4]:
def check_feasibility(t, eta, X, rho, alpha, beta ):
    K = len(eta)
    check_vec = np.zeros(len(eta))
    for k in range(K):
        num = rho*eta[k]*( X[:,k]@alpha[:,k] )**2
        den = X[:,k]@alpha[:,k] - rho * ( X[:,k]@np.multiply(alpha[:,k],beta[:,k]) )* eta[k] # removing the k-th element so we can avoid creating a new list
        for k1 in range(K):
            den += rho * ( X[:,k]@np.multiply(alpha[:,k],beta[:,k1]) )* eta[k1] 
        check_vec[k] = int( num >= den*t )
    return np.sum(check_vec) == K

Now we fix $t$ and $X$ and try to cast and solve a convex feasibility problem with 'cvxpy'.

In [5]:
eta = cp.Variable(K, nonneg = True)
G = cp.Parameter((K,K))
d = cp.Parameter(K)
prob = cp.Problem(cp.Minimize(0),
                    [G@eta - d >= 0,
                    eta <= np.ones(K)])

print("prob is DCP:", prob.is_dcp())

prob is DCP: True


In [6]:
# initialize parameters
def initialize_feas_params(rho, X, alpha, beta, t):
    
    K = len(alpha[1,:])
    dtmp = np.zeros(K)
    Gtmp = np.zeros((K,K))
    
    for k in range(K): # for row
        dtmp[k] = X[:,k].dot(alpha[:,k])
        Gtmp[k,k] = rho/t * dtmp[k]**2
        for k1 in range(K): # for column (excluding diagonal)
            if k1 != k:
                Gtmp[k,k1] = -rho * X[:,k].dot( np.multiply(alpha[:,k],beta[:,k1]) )
    
    
    return [Gtmp,dtmp]

In [7]:
tol = 10**(-7) # tolerance for the bisection algorithm  
maxiter = 100

In [8]:
# Bisection algorithm
tmin = 0
tmax = 1

it = 0

while (tmax-tmin)>tol and it < maxiter :
    it = it+1
    t = (tmin + tmax)/2
    
    G.value, d.value = initialize_feas_params(rho, X_mat, alpha, beta, t)
    
    
    prob.solve()
    print('Status: ', prob.status)
    if prob.status=='infeasible':
        tmax = t
    else:
        tmin = t



Status:  infeasible
Status:  optimal
Status:  infeasible
Status:  optimal
Status:  infeasible
Status:  optimal
Status:  optimal
Status:  infeasible
Status:  infeasible
Status:  optimal
Status:  infeasible
Status:  infeasible
Status:  optimal
Status:  optimal
Status:  infeasible
Status:  infeasible
Status:  infeasible
Status:  optimal
Status:  optimal
Status:  optimal
Status:  optimal
Status:  infeasible
Status:  infeasible
Status:  infeasible


In [9]:
G.value, d.value = initialize_feas_params(rho, X_mat, alpha, beta, tmin)
prob.solve()
print('Status: ', prob.status)
eta_opt = prob.variables()
print('Solution: ', eta.value)

Status:  optimal
Solution:  [0.79411479 0.83571152 0.99999992]
